In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
model = "EFFNETV2S"
#dset = "FastFoodV2"
dset = "PlantVillage"
model_set = model + "_" + dset
#last_epoch = 9
last_epoch = 14

In [ ]:
classes = ['Baked Potato', 'Burger', 'Crispy Chicken', 'Donut', 'Fries', 'Hot Dog', 'Pizza', 'Sandwich', 'Taco', 'Taquito']
cnts = [1500, 1164, 1404,  513, 1500,  924,  136, 1301, 1500, 1198]

In [ ]:
classes=['Pepper__bell___Bacterial_spot',
         'Pepper__bell___healthy',
         'Potato___Early_blight', 
         'Potato___Late_blight',
         'Potato___healthy',
        'Tomato_Bacterial_spot', 
        'Tomato_Early_blight',
        'Tomato_Late_blight', 
        'Tomato_Leaf_Mold',
        'Tomato_Septoria_leaf_spot',
        'Tomato_Spider_mites_Two_spotted_spider_mite',
        'Tomato__Target_Spot', 
        'Tomato__Tomato_YellowLeaf__Curl_Virus',
        'Tomato__Tomato_mosaic_virus', 
        'Tomato_healthy']
cnts=[897,1330,900,900,137,1914,900,1718,857,1594,1508,1264,2887,336,1432]

In [ ]:
sum(cnts)

In [ ]:
for lab,cnt in zip(classes, cnts):
    if cnt <500:
        size="small"
    elif cnt<1000:
        size="medium"
    elif cnt<2000:
        size="big"
    else:
        size="super big"
    print(lab, size, cnt)

In [ ]:
base = pd.read_csv(f"../logs/{model_set}_base.csv")
base_pc = pd.read_csv(f"../logs/{model_set}_base_perclass.csv")
df = pd.read_csv(f"../logs/{model_set}.csv")
df_pc = pd.read_csv(f"../logs/{model_set}_perclass.csv")

In [ ]:
base.loc[base.epoch==last_epoch].mean()

In [ ]:
measures = ["test acc", "test loss", "test f1", "weighted f1"]
measure_labels = ["Dokładność [%]", "Strata", "Dokładność F1 (globalna)", "Dokładność F1 (średnia ważona)"]
heuristic_names = {"least_confidence": "Najmniejsza Pewność" , "smallest_margin":"Najmniejszy Margines", "largest_margin": "Największy Margines", "representative_sampling":"Próbkowanie Reprezentatywne (PR)", 
                   "mc_dropout":"Monte Carlo Dropout (MCD)", "representative_mc_dropout":"PR + MCD", "None":"Wybór Losowy"}

In [ ]:
base_melted = pd.melt(base.groupby("epoch").mean()[["val f1", "val loss", "test f1", "test loss"]].reset_index(), value_vars=["val f1", "test f1", "val loss", "test loss"], id_vars="epoch")

In [ ]:
base_std = pd.melt(base.groupby("epoch").std()[["val f1", "val loss", "test f1", "test loss"]].reset_index(), value_vars=["val f1", "test f1", "val loss", "test loss"], id_vars="epoch")

In [ ]:
import plotly.express as px
import plotly.graph_objs as go

def line(error_y_mode=None,**kwargs):
    """Extension of `plotly.express.line` to use error bands."""
    ERROR_MODES = {'bar','band','bars','bands',None}
    if error_y_mode not in ERROR_MODES:
        raise ValueError(f"'error_y_mode' must be one of {ERROR_MODES}, received {repr(error_y_mode)}.")
    if error_y_mode in {'bar','bars',None}:
        fig = px.line(**kwargs)
    elif error_y_mode in {'band','bands'}:
        if 'error_y' not in kwargs:
            raise ValueError(f"If you provide argument 'error_y_mode' you must also provide 'error_y'.")
        figure_with_error_bars = px.line(**kwargs)
        fig = px.line(**{arg: val for arg,val in kwargs.items() if arg != 'error_y'})
        
        
        for data in figure_with_error_bars.data:
            x = list(data['x'])
            y_upper = list(data['y'] + data['error_y']['array'])
            y_lower = list(data['y'] - data['error_y']['array'] if data['error_y']['arrayminus'] is None else data['y'] - data['error_y']['arrayminus'])
            i=0
            my_str = data['line']['color'].lstrip("(rgb").replace(")","")
            my_list = my_str.split(",")
            number1 = int(my_list[0])
            number2 = int(my_list[1])
            number3 = int(my_list[2])
            color = f"rgba({number1}, {number2}, {number3}, 0.3)"
            #color = f"rgba({tuple(int(data['line']['color'].lstrip('#')[i:i+2], 16) for i in (0, 2, 4))},.3)".replace('((','(').replace('),',',').replace(' ','')
            fig.add_trace(
                go.Scatter(
                    x = x+x[::-1],
                    y = y_upper+y_lower[::-1],
                    fill = 'toself',
                    fillcolor = color,
                    line = dict(
                        color = 'rgba(255,255,255,0)'
                    ),
                    hoverinfo = "skip",
                    showlegend = False,
                    legendgroup = data['legendgroup'],
                    xaxis = data['xaxis'],
                    yaxis = data['yaxis'],
                )
            )
        reordered_data = []
        for i in range(int(len(fig.data)/2)):
            reordered_data.append(fig.data[i+int(len(fig.data)/2)])
            reordered_data.append(fig.data[i])
        fig.data = tuple(reordered_data)
    return fig

In [ ]:
error_y_mode = "bands"

In [ ]:
legend_labels = {"val f1": "Walidacyjny", "test f1":"Testowy", "val loss": "Walidacyjny", "test loss":"Testowy"}
for m, l in zip(("f1", "loss"),("Dokładność F1", "Strata")):
    fig = px.line(
            data_frame=base_melted.loc[base_melted.variable.str.contains(m)], 
            x = "epoch",
            y = "value",
            error_y=base_std.loc[base_std.variable.str.contains(m)]["value"],
            #error_y_mode=error_y_mode,
            color = "variable",
            color_discrete_sequence=[
                # 'rgb(127, 60, 141)',
#  'rgb(17, 165, 121)',
 'rgb(57, 105, 172)',
#  'rgb(242, 183, 1)',
#  'rgb(231, 63, 116)',
 'rgb(128, 186, 90)',
 'rgb(230, 131, 16)',
#  'rgb(0, 134, 149)',
#  'rgb(207, 28, 144)',
#  'rgb(249, 123, 114)',
#  'rgb(165, 170, 153)',
 'rgb(64,64,64)'],
            # color_discrete_sequence = px.colors.qualitative.Bold,
            markers=True,
            width=800,
            height = 600,
            render_mode="svg",
            labels = {"variable": "Zbiór", "epoch":"Epoka", "value": l},
        )
    
    fig.for_each_trace(lambda t: t.update(name = legend_labels[t.name],
                                        legendgroup = legend_labels[t.name],
                                        hovertemplate = t.hovertemplate.replace(t.name, legend_labels[t.name])
                                        )
                    )
    fontsize =15
    
    fig.update_layout(
        plot_bgcolor='white',
    )
    fig.update_xaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey',
        titlefont = dict(size = fontsize),
        tickfont = dict(size = fontsize),
    )
    fig.update_yaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey',
        titlefont = dict(size = fontsize),
        tickfont = dict(size = fontsize),
    )
   
    if "loss" in m:
        fig.update_layout(legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99 ,
            bordercolor="Black",
            borderwidth=1), font=dict(size=fontsize-1), legend_title_font=dict(size=fontsize-1))
    else:
        fig.update_layout(legend=dict(
            yanchor="bottom",
            y=0.01,
            xanchor="right",
            x=0.99 ,
            bordercolor="Black",
            borderwidth=1
        ), font=dict(size=fontsize-1), legend_title_font=dict(size=fontsize-1))
        # fig.show()
    fig.write_image(f"../imgs/{model_set}_{m.replace(' ','_')}_val_test.png")
    fig.show()

In [ ]:
grouped_base = base.loc[base.epoch==last_epoch, measures].mean()
grouped_base["test acc"] = grouped_base["test acc"]*100

In [ ]:
grouped_df = df.loc[df.epoch == last_epoch].groupby(["active_iter", "heuristic"])[measures].mean()
grouped_df["test acc"] = grouped_df["test acc"]*100
grouped_df = grouped_df.reset_index()
grouped_df["active_iter"]=(grouped_df["active_iter"]*0.1 + 0.1)*100

In [ ]:
grouped_df_std = df.loc[df.epoch == last_epoch].groupby(["active_iter", "heuristic"])[measures].std()
grouped_df_std["test acc"] = grouped_df_std["test acc"]*100
grouped_df_std = grouped_df_std.reset_index()
grouped_df_std["active_iter"]=(grouped_df_std["active_iter"]*0.1 + 0.1)*100

In [ ]:
error_y_mode="bands"

In [ ]:

for m,l in zip(measures, measure_labels):
    fig = line(
        data_frame = grouped_df.reset_index(),
        x = "active_iter",
        y = m,
        error_y=grouped_df_std[m],
        error_y_mode=error_y_mode,
        color = "heuristic",
        # color_discrete_sequence = px.colors.qualitative.Bold,
        color_discrete_sequence=['rgb(127, 60, 141)',
 'rgb(17, 165, 121)',
 'rgb(57, 105, 172)',
 'rgb(242, 183, 1)',
 'rgb(231, 63, 116)',
 'rgb(128, 186, 90)',
 'rgb(230, 131, 16)',
#  'rgb(0, 134, 149)',
#  'rgb(207, 28, 144)',
#  'rgb(249, 123, 114)',
#  'rgb(165, 170, 153)',
 'rgb(64,64,64)'],
        markers=True,
        width=800,
        height = 600,
        category_orders={
            "heuristic":["least_confidence", "smallest_margin", "largest_margin", "representative_sampling", "mc_dropout", "representative_mc_dropout", None]
        },
        labels = {"heuristic": "Heurystyka", "active_iter":"Rozmiar zbioru", m:l},
        render_mode="svg"
    )
    '''fig.for_each_trace(lambda t: t.update(name = heuristic_names[t.name],
                                      legendgroup = heuristic_names[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, heuristic_names[t.name])
                                     )
                  )'''
    fontsize =15
    fig.add_hline(y=grouped_base[m], line_dash="dash", line_width=1, annotation_text="Wartość bazowa", 
              annotation_position="bottom left", annotation_font = dict(size = fontsize),)
    fig.update_layout(
        plot_bgcolor='white',
    )
    fig.update_xaxes(
        mirror=True,
        ticks='outside',
        ticksuffix="%",
        showline=True,
        linecolor='black',
        gridcolor='lightgrey',
        titlefont = dict(size = fontsize),
        tickfont = dict(size = fontsize),
    )
    fig.update_yaxes(
        mirror=True,
        ticks='outside',
        showline=True,
        linecolor='black',
        gridcolor='lightgrey',
        titlefont = dict(size = fontsize),
        tickfont = dict(size = fontsize),
    )
    if "loss" in m:
        fig.update_layout(legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.99 ,
            bordercolor="Black",
            borderwidth=1), font=dict(size=fontsize-1), legend_title_font=dict(size=fontsize-1))
    else:
        fig.update_layout(legend=dict(
            yanchor="bottom",
            y=0.01,
            xanchor="right",
            x=0.99 ,
            bordercolor="Black",
            borderwidth=1
        ), font=dict(size=fontsize-1), legend_title_font=dict(size=fontsize-1))
    fig.show()
    # break
    fig.write_image(f"../imgs/{model_set}_{m.replace(' ','_')}.png")

In [ ]:
grouped_df_pc = df_pc.loc[df_pc.epoch == last_epoch].groupby(["active_iter", "heuristic", "class"])[["test acc", "test f1",]].mean()
grouped_df_pc["test acc"] = grouped_df_pc["test acc"]*100
grouped_df_pc = grouped_df_pc.reset_index()
grouped_df_pc["active_iter"]=(grouped_df_pc["active_iter"]*0.1 + 0.1)*100

In [ ]:
grouped_df_pc_std = df_pc.loc[df_pc.epoch == last_epoch].groupby(["active_iter", "heuristic", "class"])[["test acc", "test f1",]].std()
grouped_df_pc_std["test acc"] = grouped_df_pc_std["test acc"]*100
grouped_df_pc_std = grouped_df_pc_std.reset_index()
grouped_df_pc_std["active_iter"]=(grouped_df_pc_std["active_iter"]*0.1 + 0.1)*100

In [ ]:
grouped_base_pc = base_pc.loc[base_pc.epoch == last_epoch].groupby(["class"])[["test acc", "test f1",]].mean()
grouped_base_pc["test acc"] = grouped_base_pc["test acc"]*100


In [ ]:
grouped_base_pc

In [ ]:
grouped_df_pc.reset_index()["class"].unique()

In [ ]:
for c, cnt in zip(grouped_df_pc.reset_index()["class"].unique(), cnts):
    print(c)
    for m,l in zip(["test acc", "test f1",],["Dokładność [%]", "Dokładność F1"]):
        fig = line(
            data_frame = grouped_df_pc.reset_index().loc[grouped_df_pc.reset_index()["class"]==c].reset_index(),
            x = "active_iter",
            y = m,
            error_y = grouped_df_pc_std.reset_index().loc[grouped_df_pc_std.reset_index()["class"]==c].reset_index()[m],
            error_y_mode="bands",
            color = "heuristic",
            # color_discrete_sequence = px.colors.qualitative.Bold,
            color_discrete_sequence=['rgb(127, 60, 141)',
 'rgb(17, 165, 121)',
 'rgb(57, 105, 172)',
 'rgb(242, 183, 1)',
 'rgb(231, 63, 116)',
 'rgb(128, 186, 90)',
 'rgb(230, 131, 16)',
#  'rgb(0, 134, 149)',
#  'rgb(207, 28, 144)',
#  'rgb(249, 123, 114)',
#  'rgb(165, 170, 153)',
 'rgb(64,64,64)'],
            markers=True,
            width=800,
            height = 600,
            category_orders={
                "heuristic":["least_confidence", "smallest_margin", "largest_margin", "representative_sampling", "mc_dropout", "representative_mc_dropout", None]
            },
            labels = {"heuristic": "Heurystyka", "active_iter":"Rozmiar zbioru", m:l},
            render_mode="svg",
            # title = f"{c} ({cnt})"
        )
        fontsize =15
        '''fig.for_each_trace(lambda t: t.update(name = heuristic_names[t.name],
                                        legendgroup = heuristic_names[t.name],
                                        hovertemplate = t.hovertemplate.replace(t.name, heuristic_names[t.name])
                                        ))'''
        fig.add_hline(y=grouped_base_pc.loc[c,m], line_dash="dash", line_width=1, annotation_text="Wartość bazowa", 
                annotation_position="bottom left",annotation_font = dict(size = fontsize),)
        fig.update_layout(
            plot_bgcolor='white',
            title=dict(
                yanchor="top",
                y=0.9,
                xanchor="left",
                x=0.1 ,
            )
        )
        fig.update_xaxes(
            mirror=True,
            ticks='outside',
            ticksuffix="%",
            showline=True,
            linecolor='black',
            gridcolor='lightgrey',
            titlefont = dict(size = fontsize),
        tickfont = dict(size = fontsize),
        )
        fig.update_yaxes(
            mirror=True,
            ticks='outside',
            showline=True,
            linecolor='black',
            gridcolor='lightgrey',
            titlefont = dict(size = fontsize),
        tickfont = dict(size = fontsize),
        )
        if "loss" in m:
            fig.update_layout(legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="right",
                x=0.99 ,
                bordercolor="Black",
                borderwidth=1),
                font=dict(size=fontsize-2), legend_title_font=dict(size=fontsize-3))
        else:
            fig.update_layout(legend=dict(
                yanchor="bottom",
                y=0.01,
                xanchor="right",
                x=0.99 ,
                bordercolor="Black",
                borderwidth=1,
                # fontsize=fontsize-3
            ), font=dict(size=fontsize-2), legend_title_font=dict(size=fontsize-2))
        fig.show()
    #     break
    # break
        fig.write_image(f"../imgs/{model_set}_{m.replace(' ','_')}_{c.replace(' ','_')}.png")